## Diese Datei wurde fuer das erstellen aller .pkl Dateien im Ordner pickle benutzt

In [ ]:
pip install impyute

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from mypackage.MyMethods import *
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

import pickle
import secrets
import pandas as pd
import numpy as np
import impyute as impy

In [ ]:
#um komplette DataFrames auszugeben
pd.set_option("display.max_rows", None, "display.max_columns", None)
np.set_printoptions(threshold=np.inf)

In [ ]:
d1 = pd.read_csv("datasets/generated_data2.csv")
d1 = d1[["object_id", "time", "feature1"]]
d1 = d1.assign(feature1_nan=d1[["feature1"]])

pickle_dfs = []
pickle_avg_distance = []
pickle_best_rating = []

secure_random = secrets.SystemRandom()
z = StandardScaler()

In [ ]:
for rota in range(1, 2):  #bestimmt wie viele gepickelte Dateien man haben möchte
    for loops in range(1, 3): #bestimmt die Anzahl an DataFrames in einer gepickelten Datei
        try:
            d1 = d1[["object_id", "time", "feature1", "feature1_nan"]]
            ix = [(row, d1.columns.get_loc("feature1_nan")) for row in d1[d1["feature1_nan"].notna()].index]
            sample_ix = secure_random.sample(ix, 158) # zweiter parameter bestimmt wie viele Datenpunkte pro Datensatz fehlen sollen

            for row, col in sample_ix:
                d1.iat[row, d1.columns.get_loc("feature1_nan")] = np.nan

            #Nicht Vorhandene Werte von Zeitreihen simulieren
            sample_df = d1[["object_id","time","feature1_nan"]].dropna()
            sample_df = sample_df.rename(columns={"feature1_nan" : "feature1"})
            sample_df[["time", "feature1"]] = z.fit_transform(sample_df[["time", "feature1"]])

            rating_list = calc_best_rating(sample_df, 'object_id', 'time', 'feature1', 'assignments')

            test_db1 = DBSCAN(eps = rating_list[1], min_samples = rating_list[2]).fit(sample_df[["time", "feature1"]])
            labsList = ["Noise"]
            labsList = labsList + ["Cluster " + str(i) for i in range(1, len(set(test_db1.labels_)))]

            sample_df["assignments"] = test_db1.labels_
            sample_df[["time", "feature1"]] = z.inverse_transform(sample_df[["time", "feature1"]])
            sample_df["time"] = sample_df["time"].round()  #um rundungsfehler nach den scalen zu umgehen

            #Erstellen aller DataFrames zu den dazugehoerigen Methoden. Interpolationsmethoden exclusive!
            mfcm_df = most_frequent_cluster_member(sample_df, "object_id", "time", "feature1", "assignments")
            mfcm_nearest_df = most_frequent_cluster_member_nearest(sample_df, "object_id", "time", "feature1", "assignments")
            ppa_df = pre_and_post_clustering_analysis(sample_df, "object_id", "time", "feature1", "assignments")
            new_method_mean_df = new_method_mean(sample_df, "object_id", "time", "feature1", "assignments")
            new_method_median_df = new_method_median(sample_df, "object_id", "time", "feature1", "assignments")
            new_method_mode_df = new_method_mode(sample_df, "object_id", "time", "feature1", "assignments")
            mean_timestemp_df = mean_timestemp(sample_df, "object_id", "time", "feature1")
            median_timestemp_df = median_timestemp(sample_df, "object_id", "time", "feature1")
            mean_timeseries_df = mean_timeseries(sample_df, "object_id", "time", "feature1")
            median_timeseries_df = median_timeseries(sample_df, "object_id", "time", "feature1")
            knn_df = pd.DataFrame(impy.fast_knn(d1[["object_id","time","feature1_nan"]].values, k=1), columns=["object_id","time","fast_knn"])
        except:
            d1 = d1[["object_id", "time", "feature1"]]
            d1 = d1.assign(feature1_nan=d1[["feature1"]])
            continue

        #um eine neue DataFrame an unser bestehendes DataFrames zu hängen
        mfcm_df["time"] = mfcm_df["time"].astype(int)   #wegen float und int fehlern beim mergen
        d1 = pd.merge(d1, mfcm_df, how='left', on=["time", "object_id"])
        d1.mfcm.fillna(d1.feature1_nan, inplace=True)

        ppa_df["time"] = ppa_df["time"].astype(int)
        d1 = pd.merge(d1, ppa_df, how='left', on=["time", "object_id"])
        d1.ppa.fillna(d1.feature1_nan, inplace=True)

        mfcm_nearest_df["time"] = mfcm_nearest_df["time"].astype(int)
        d1 = pd.merge(d1, mfcm_nearest_df, how='left', on=["time", "object_id"])
        d1.mfcm_nearest.fillna(d1.feature1_nan, inplace=True)

        new_method_mean_df["time"] = new_method_mean_df["time"].astype(int)
        d1 = pd.merge(d1, new_method_mean_df, how='left', on=["time", "object_id"])
        d1.new_method_mean.fillna(d1.feature1_nan, inplace=True)

        new_method_median_df["time"] = new_method_median_df["time"].astype(int)
        d1 = pd.merge(d1, new_method_median_df, how='left', on=["time", "object_id"])
        d1.new_method_median.fillna(d1.feature1_nan, inplace=True)

        new_method_mode_df["time"] = new_method_mode_df["time"].astype(int)
        d1 = pd.merge(d1, new_method_mode_df, how='left', on=["time", "object_id"])
        d1.new_method_mode.fillna(d1.feature1_nan, inplace=True)

        mean_timestemp_df["time"] = mean_timestemp_df["time"].astype(int)
        d1 = pd.merge(d1, mean_timestemp_df, how='left', on=["time", "object_id"])
        d1.mean_timestemp.fillna(d1.feature1_nan, inplace=True)

        median_timestemp_df["time"] = median_timestemp_df["time"].astype(int)
        d1 = pd.merge(d1, median_timestemp_df, how='left', on=["time", "object_id"])
        d1.median_timestemp.fillna(d1.feature1_nan, inplace=True)

        mean_timeseries_df["time"] = mean_timeseries_df["time"].astype(int)
        d1 = pd.merge(d1, mean_timeseries_df, how='left', on=["time", "object_id"])
        d1.mean_timeseries.fillna(d1.feature1_nan, inplace=True)

        median_timeseries_df["time"] = median_timeseries_df["time"].astype(int)
        d1 = pd.merge(d1, median_timeseries_df, how='left', on=["time", "object_id"])
        d1.median_timeseries.fillna(d1.feature1_nan, inplace=True)
        
        #fast_knn
        knn_df["time"] = knn_df["time"].astype(int)
        d1 = pd.merge(d1, knn_df, how='left', on=["time", "object_id"])
        d1.fast_knn.fillna(d1.feature1_nan, inplace=True)
        
        #Fill-Mean und Fill-Median
        d1 = d1.assign(FillMean=d1.feature1_nan.fillna(d1.feature1_nan.mean()))
        d1 = d1.assign(FillMedian=d1.feature1_nan.fillna(d1.feature1_nan.median()))
        
        ids = list(set(getattr(d1,"object_id")))
        interpolation_list = []
        ##AKIMA INTERPOLATION
        for object_id in ids:
            time_series = d1[d1["object_id"] == object_id]
            time_series[["time", "feature1_nan"]] = z.fit_transform(time_series[["time", "feature1_nan"]])
            time_series = time_series.set_index('time')
            try:    
                our_result = time_series.assign(InterpolateAkima = time_series.feature1_nan.interpolate(method='akima'))
                our_result = our_result.reset_index("time")
                our_result[["time", "InterpolateAkima"]] = z.inverse_transform(our_result[["time", "InterpolateAkima"]])
                interpolation_list.append(our_result[["InterpolateAkima", "object_id", "time"]])
            except Exception as e:
                time_series = time_series.reset_index("time")
                time_series[["time", "feature1_nan"]] = z.inverse_transform(time_series[["time", "feature1_nan"]])
                time_series = time_series.rename(columns={"feature1_nan" : "InterpolateAkima"})
                interpolation_list.append(time_series[["InterpolateAkima", "object_id", "time"]])
                pass
        d1 = pd.merge(d1, pd.concat(interpolation_list) , how='left', on=["time", "object_id"])

        
        ##LINEAR INTERPOLATION
        interpolation_list.clear()
        for object_id in ids:
            time_series = d1[d1["object_id"] == object_id]
            time_series[["time", "feature1_nan"]] = z.fit_transform(time_series[["time", "feature1_nan"]])
            time_series = time_series.set_index('time')
            try:
                our_result = time_series.assign(InterpolateLinear=time_series.feature1_nan.interpolate(method='linear'))
                our_result = our_result.reset_index("time")
                our_result[["time", "InterpolateLinear"]] = z.inverse_transform(our_result[["time", "InterpolateLinear"]])
                interpolation_list.append(our_result[["InterpolateLinear",  "object_id", "time"]])
            except Exception as e:
                time_series = time_series.reset_index("time")
                time_series[["time", "feature1_nan"]] = z.inverse_transform(time_series[["time", "feature1_nan"]])
                time_series = time_series.rename(columns={"feature1_nan" : "InterpolateLinear"})
                interpolation_list.append(time_series[["InterpolateLinear", "object_id", "time"]])
                pass
        d1 = pd.merge(d1, pd.concat(interpolation_list) , how='left', on=["time", "object_id"])

        
        ##QUADRATIC SPLINE INTERPOLATION
        interpolation_list.clear()
        for object_id in ids:
            time_series = d1[d1["object_id"] == object_id]
            time_series[["time", "feature1_nan"]] = z.fit_transform(time_series[["time", "feature1_nan"]])
            time_series = time_series.set_index('time')
            try:
                our_result = time_series.assign(InterpolateQuadratic=time_series.feature1_nan.interpolate(method='quadratic'))
                our_result = our_result.reset_index("time")
                our_result[["time", "InterpolateQuadratic"]] = z.inverse_transform(our_result[["time", "InterpolateQuadratic"]])
                interpolation_list.append(our_result[["InterpolateQuadratic", "object_id", "time"]])
            except Exception as e:
                time_series = time_series.reset_index("time")
                time_series[["time", "feature1_nan"]] = z.inverse_transform(time_series[["time", "feature1_nan"]])
                time_series = time_series.rename(columns={"feature1_nan" : "InterpolateQuadratic"})
                interpolation_list.append(time_series[["InterpolateQuadratic", "object_id", "time"]])
                pass
        d1 = pd.merge(d1, pd.concat(interpolation_list) , how='left', on=["time", "object_id"])
        
        ##CUBIC SPLINE INTERPOLATION
        interpolation_list.clear()
        for object_id in ids:
            time_series = d1[d1["object_id"] == object_id]
            time_series[["time", "feature1_nan"]] = z.fit_transform(time_series[["time", "feature1_nan"]])
            time_series = time_series.set_index('time')
            try:
                our_result = time_series.assign(InterpolateCubic= time_series.feature1_nan.interpolate(method='cubic'))
                our_result = our_result.reset_index("time")
                our_result[["time", "InterpolateCubic"]] = z.inverse_transform(our_result[["time", "InterpolateCubic"]])
                interpolation_list.append(our_result[["InterpolateCubic", "object_id", "time"]])
            except Exception as e:
                time_series = time_series.reset_index("time")
                time_series[["time", "feature1_nan"]] = z.inverse_transform(time_series[["time", "feature1_nan"]])
                time_series = time_series.rename(columns={"feature1_nan" : "InterpolateCubic"})
                interpolation_list.append(time_series[["InterpolateCubic", "object_id", "time"]])
                pass
        d1 = pd.merge(d1, pd.concat(interpolation_list) , how='left', on=["time", "object_id"])
        

        ## SPLINE INTERPOLATION GRAD 5
        interpolation_list.clear()
        for object_id in ids:
            time_series = d1[d1["object_id"] == object_id]
            time_series[["time", "feature1_nan"]] = z.fit_transform(time_series[["time", "feature1_nan"]])
            time_series = time_series.set_index('time')
            try:
                our_result = time_series.assign(InterpolatePoly5=time_series.feature1_nan.interpolate(method='polynomial', order=5))
                our_result = our_result.reset_index("time")
                our_result[["time", "InterpolatePoly5"]] = z.inverse_transform(our_result[["time", "InterpolatePoly5"]])
                interpolation_list.append(our_result[["InterpolatePoly5", "object_id", "time"]])
            except Exception as e:
                time_series = time_series.reset_index("time")
                time_series[["time", "feature1_nan"]] = z.inverse_transform(time_series[["time", "feature1_nan"]])
                time_series = time_series.rename(columns={"feature1_nan" : "InterpolatePoly5"})
                interpolation_list.append(time_series[["InterpolatePoly5", "object_id", "time"]])
                pass
        d1 = pd.merge(d1, pd.concat(interpolation_list) , how='left', on=["time", "object_id"])
        
        methods = list(d1)
        delete_rows = ["feature1","feature1_nan","time","object_id"]
        for i in delete_rows:
            methods.remove(i)

        avg = [(method, avg_distance(d1.feature1, d1[method], d1[d1.feature1_nan.isna()].index)) for method in methods]
        avg.sort(key=lambda x: x[1])

        pickle_dfs.append(d1)
        pickle_avg_distance.append(avg)
        pickle_best_rating.append([rating_list[0], rating_list[1], rating_list[2]])
        d1 = d1[["object_id", "time", "feature1"]]
        d1 = d1.assign(feature1_nan=d1[["feature1"]])
        print(loops)

    pidfs = 'pickle/pickle_df_158_500.pkl'
    with open(pidfs, 'wb') as pickle_file1:
        pickle.dump(pickle_dfs, pickle_file1)

    piavg = 'pickle/pickle_avg_158_500.pkl'
    with open(piavg, 'wb') as pickle_file2:
        pickle.dump(pickle_avg_distance, pickle_file2)

    pira = 'pickle/pickle_rating_158_500.pkl'
    with open(pira, 'wb') as pickle_file3:
        pickle.dump(pickle_best_rating, pickle_file3)

    d1 = d1[["object_id", "time", "feature1"]]
    d1 = d1.assign(feature1_nan=d1[["feature1"]])
    pickle_dfs.clear()
    pickle_avg_distance.clear()
    pickle_best_rating.clear()
    print(rota)